In [1]:
import pandas as pd
import numpy as np

In [2]:
df_news = pd.read_csv('news.csv')

In [3]:
df_news


,text,label
0,Помощник депутата Палаты представителей: мы да...,1
1,"Вы когда-нибудь чувствовали, что ваша жизнь дв...",0
2,Почему правда может привести к увольнению 29 о...,1
3,"Выявлено 15 мирных жителей, погибших в результ...",1
4,Распечатать\nИранская женщина была приговорена...,1
...,...,...
14995,САКРАМЕНТО. Когда Берни Сандерс провел митинг ...,0
14996,В конце времен врагами человека станут его дом...,1
14997,"ГОНКОНГ. Сотни гриндов, заплывших в мелководну...",0
14998,20:20\n( ͡° ͜ʖ ͡°)\n20:33\nHaloPro2121\nсмешно...,1


In [4]:
df_news.label.value_counts()

label
0    8324
1    6676
Name: count, dtype: int64

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pymorphy2

nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
nltk_tokenizer = RegexpTokenizer(r'[а-яёa-z]+')

morph = pymorphy2.MorphAnalyzer()


[nltk_data] Downloading package stopwords to /home/timur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def text_preprocessing(text):
    words = nltk_tokenizer.tokenize(text.lower())
    lem_text = [morph.parse(w)[0].normal_form for w in words if w not in stop_words]

    return lem_text

In [7]:
import gensim
import os

if not os.path.isfile('220.zip'):
  !wget http://vectors.nlpl.eu/repository/20/220.zip
  !unzip 220.zip

w2v = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [25]:
import wget

In [26]:
import ufal.udpipe as udp
import corpy.udpipe as crp

# Скачивание модели UDPipe, обученную на русском языке
udp_model_url = r'https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe'
udp_model_filename = udp_model_url.rsplit('/', 1)[-1]
if not os.path.isfile(udp_model_filename):
    wget.download(udp_model_url)

# Загрузка модели в оболочку corpy
corpy_model = crp.Model(udp_model_filename)



In [27]:
# Функция для тегирования слов
def udp_tagging(lem_text):
    sents = [list(corpy_model.process(w)) for w in lem_text]
    tagged_words = [s[0].words[1].form + '_' + s[0].words[1].upostag for s in sents if s]

    return tagged_words

In [ ]:
X_not_filtered_texts = df_news['text'].apply(lambda x: udp_tagging(text_preprocessing(x))).array


In [ ]:
X_filtered_texts = []
for text in X_not_filtered_texts:
    words = []
    for word in text:
        if word in w2v.vocab:
            words.append(word)
        else:
            pass
    X_filtered_texts.append(words)

In [ ]:
from sklearn.model_selection import train_test_split

X = np.array([list(map(lambda x: w2v.vocab[x].index, t)) for t in X_filtered_texts]).astype('float32')

Y = np.array(df_news['label']).astype('float32').reshape((-1,1))

x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
from tensorflow.keras.models import

from tensorflow.keras.layers import Dense, Dropout

seq_model = Sequential()

seq_model.add(w2v.wv.get_keras_embedding(train_embeddings=False))
seq_model.add(Dense(50, activation='relu'))
seq_model.add(Dropout(0.6))
seq_model.add(Dense(1, activation='sigmoid'))

seq_model.compile(loss='mean_squared_logarithmic_error',
                  optimizer='adam', metrics=['accuracy'])

In [ ]:
seq_model.fit(x_train, y_train, epochs=20,
              validation_data=(x_test, y_test))

In [ ]:
fake_text = 'Центробанк России готовится заморозить вклады населения…'

fake_preproc = udp_tagging(text_preprocessing(fake_text))

fake_words_tagged = []
for word in fake_preproc:
    if word in w2v.vocab:
    fake_words_tagged.append(word)

fake_indexed = np.array(list(map(lambda x: w2v.vocab[x].index, fake_words_tagged))).astype('float32')

seq_model.predict(fake_indexed)

In [ ]:
import joblib

# Сохранение модели
joblib.dump(seq_model, 'model.pkl')

# Загрузка сохраненной модели
loaded_model = joblib.load('model.pkl')